# Describing a Lab Protocol from Video


In [30]:
from IPython.display import display, Image as PyImage
from io import BytesIO
import base64
import time

from moviepy.editor import VideoFileClip
from PIL import Image
from openai import OpenAI


In [21]:
client = OpenAI()

def resize(img, size=512):
    original_width, original_height = img.size
    aspect_ratio = original_width / original_height

    if original_width < original_height:
        new_width = size
        new_height = int(new_width / aspect_ratio)
    else:
        new_height = size
        new_width = int(new_height * aspect_ratio)

    return img.resize((new_width, new_height), Image.BICUBIC)

def extract_data(filename, frameskip=10):
    video = VideoFileClip(filename)

    # Extract frames
    frames = []
    for i, frame in enumerate(video.iter_frames()):
        if i % frameskip == 0:
            frame_image = Image.fromarray(frame).resize((512,512), Image.BICUBIC)
            im_file = BytesIO()
            frame_image.save(im_file, format="JPEG")
            im_bytes = im_file.getvalue()
            frames.append(base64.b64encode(im_bytes).decode('utf-8'))

    # Extract audio
    audio = video.audio
    audio_file = "extracted_audio.mp3"
    audio.write_audiofile(audio_file)

    return frames, audio_file

def transcribe(audio_filepath):
    transcript = client.audio.transcriptions.create(
        file=open(audio_filepath, "rb"),
        model="whisper-1",
        # prompt=prompt,
    )
    return transcript.text

def approx_cost(frames, transcript, token_output=300):
    image_cost = 0.00765 * len(frames)
    transcript_cost = 0.01 / 1000 * len(transcript.split()) * 1.4
    output_cost = token_output * 0.03 / 1000
    return f"Images: {image_cost}\nTranscript: {transcript_cost}\nOutput: {output_cost}\nTotal: {image_cost + transcript_cost + output_cost}"


Display frames to make sure we've read them in correctly:


In [40]:
# Load your video
video_file = "dynamic-video.mp4"
frames, audiofile = extract_data(video_file, frameskip=60)
print(f"N frames: {len(frames)}")


MoviePy - Writing audio in extracted_audio.mp3


MoviePy - Done.
N frames: 35


In [ ]:
display_handle = display(None, display_id=True)
for img in frames:
    display_handle.update(PyImage(data=base64.b64decode(img.encode("utf-8"))))
    time.sleep(0.2)


In [42]:
audio_transcription = transcribe(audiofile)
audio_transcription


"I'm picking up a pipette, taking a tip, aspirating 25 microliters from A1 and dispensing it into A9. I am threshing my tip and switching pipette. I am going to take a new tip, aspirate 1 microliter from well A3 and dispensing it into well A9. I am threshing the tip, I am picking up a new tip, I am going to aspirate 1 microliter from well A5 and dispensing it into well A9. I am threshing the tip and I'm switching pipettes. I pick up a new tip, I'm going into well A9 and I'm going to mix by aspirating up and down a few times and I'm transferring the mix into my thermocycler. I am threshing the tip and I'm placing the pipette where we started."

In [43]:
print(f"Approximate cost: {approx_cost(frames, audio_transcription)}")


Approximate cost: Images: 0.26775
Transcript: 0.001792
Output: 0.009
Total: 0.278542


In [44]:
def make_image_message(frame):
    return {
        "type": "image_url",
        "image_url": {
            "url": f"data:image/jpeg;base64,{frame}",
            "detail": "low",
        },
    }

PROMPT_MESSAGES = [
    {
        "role": "user",
        "content": [
            {
                "type": "text",
                "text": "The following is the transcribed audio and video frames from a simple lab demonstration.",
            },
            {
                "type": "text",
                "text": f"Transcribed audio: {audio_transcription}",
            },
            {
                "type": "text",
                "text": "These are frames from a simple demonstration.",
            },
            *map(make_image_message, frames),
            {
                "type": "text",
                "text": "Turn this demonstration into a full laboratory protocol. Format it properly, but be concise.",
            },
        ],
    },
]
# PROMPT_MESSAGES[0]["content"].keys()


In [45]:
params = {
    "model": "gpt-4-vision-preview",
    "messages": PROMPT_MESSAGES,
    "max_tokens": 1000,
}

result = client.chat.completions.create(**params)
print(result.choices[0].message.content)


Title: Pipette Handling and Sample Transfer Protocol

Objective: To accurately transfer liquid samples between wells using a pipette in a laboratory setting.

Materials:
- Micropipettes capable of aspirating 1 µL and 25 µL volumes
- Sterile pipette tips for each pipette
- 96-well plate
- Thermocycler

Procedure:

1. Pipetting 25 µL:
   a. Attach a sterile pipette tip to the micropipette set to 25 µL.
   b. Depress the plunger to the first stop.
   c. Immerse the pipette tip into the solution in well A1.
   d. Slowly release the plunger to aspirate 25 µL of the solution.
   e. Move the pipette to well A9, ensuring the tip is above the well's bottom surface.
   f. Depress the plunger first to the first stop, then to the second stop to dispense the entire volume into well A9.
   g. Eject the used pipette tip into an appropriate waste container.

2. Pipetting 1 µL from Well A3:
   a. Attach a new sterile pipette tip to the micropipette set to 1 µL.
   b. Repeat steps b to g, aspirating fro